In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

stocks = requests.get('https://traderfox.de/aktien/deutschland-alle-bestandteile').content
soup = BeautifulSoup(stocks, "html.parser")
ahrefs = soup.select('#insert-stocks > tbody > tr > td.name > a')
links = [href.get('href') for href in ahrefs]
len(links)

In [3]:
import random
from time import sleep

links = open('links2.txt', 'a')

for i in range(100,200):
    stocks = requests.get(f'https://www.finanzen100.de/aktien/suche/?p={i}').content
    soup = BeautifulSoup(stocks, "html.parser")
    ahrefs = soup.select('div.card__content > div.component-group > a.InstrumentSearchTeaser')
    site_links = [href.get('href') for href in ahrefs]
    [links.write(f"{link.replace('/aktien/', '')}\n") for link in site_links]
    respect = random.randint(300,1400)/1000
    print('Site', i, 'done ... spleeping for', respect, 'ms')
    sleep(respect)

links.close()

Site 100 done ... spleeping for 0.867 ms
Site 101 done ... spleeping for 0.599 ms
Site 102 done ... spleeping for 1.243 ms
Site 103 done ... spleeping for 0.948 ms
Site 104 done ... spleeping for 0.887 ms
Site 105 done ... spleeping for 1.093 ms
Site 106 done ... spleeping for 1.097 ms
Site 107 done ... spleeping for 1.383 ms
Site 108 done ... spleeping for 1.344 ms
Site 109 done ... spleeping for 0.682 ms
Site 110 done ... spleeping for 0.365 ms
Site 111 done ... spleeping for 0.545 ms
Site 112 done ... spleeping for 0.515 ms
Site 113 done ... spleeping for 1.307 ms
Site 114 done ... spleeping for 1.138 ms
Site 115 done ... spleeping for 0.328 ms
Site 116 done ... spleeping for 1.398 ms
Site 117 done ... spleeping for 1.387 ms
Site 118 done ... spleeping for 1.004 ms
Site 119 done ... spleeping for 0.929 ms
Site 120 done ... spleeping for 1.292 ms
Site 121 done ... spleeping for 1.389 ms
Site 122 done ... spleeping for 0.579 ms
Site 123 done ... spleeping for 1.141 ms
Site 124 done ..

In [4]:
with open('links2.txt', 'r') as f:
    comp_links = f.readlines()

In [5]:
comp_links[0]

'allianz-slovenska-poistovna-as-wkn-120136_H481367056_8301381/\n'

In [6]:
from tqdm import tqdm

In [7]:
companies = []
counter = 0
for comp_link in tqdm(comp_links):
    company = {'name': comp_link.split('-wkn')[0]}
    link = comp_link.split('\n')[0]
    soup = BeautifulSoup(requests.get(f'https://www.finanzen100.de/aktien/{link}').content, "html.parser")
    try:   
        soup_summary = soup.select('div.data-listing__content')[0]
        company['country'] = soup_summary.select('p')[4].get_text()
        company['sector'] = soup_summary.select('p')[5].get_text()
    except:
        pass

    try:
        soup_perfomance = soup.select('div.card__content > div.fundamental-data')[0]
        company['2019_kgv'] = soup_perfomance.select('tr:nth-child(2) > td.column-7')[0].get_text()
        company['2020_kgv'] = soup_perfomance.select('tr:nth-child(2) > td.column-6')[0].get_text()
        company['2021_kgv'] = soup_perfomance.select('tr:nth-child(2) > td.column-5')[0].get_text()
        company['2022_kgv'] = soup_perfomance.select('tr:nth-child(2) > td.column-4')[0].get_text()
        company['2019_peg'] = soup_perfomance.select('tr:nth-child(3) > td.column-7')[0].get_text()
        company['2020_peg'] = soup_perfomance.select('tr:nth-child(3) > td.column-6')[0].get_text()
        company['2021_peg'] = soup_perfomance.select('tr:nth-child(3) > td.column-5')[0].get_text()
        company['2022_peg'] = soup_perfomance.select('tr:nth-child(3) > td.column-4')[0].get_text()
    except:
        pass

    try:
        soup_stats = soup.select('div.card__content > div.data-listing')[2].select('span.data-listing__value')
        company['EBIT-perc'] = soup_stats[0].get_text()
        company['ges_kap_rend'] = soup_stats[1].get_text()
        company['eig_kap_rend'] = soup_stats[2].get_text()
        company['umsatz'] = soup_stats[3].get_text()
        company['ums_per_empl'] = soup_stats[5].get_text()
    except:
        pass
        
    companies.append(company)
    # RESPECT!
    sleep(random.randint(200,900)/1000)

    if counter % 10 == 0:
        pd.DataFrame(companies).to_csv('comps2.csv', index=False)
    counter += 1


 38%|███▊      | 1890/4919 [52:51<1:24:42,  1.68s/it]


KeyboardInterrupt: 

In [167]:
BeautifulSoup(requests.get('https://www.finanzen100.de/aktien/apple-wkn-865985_H1526430491_86627/').content, "html.parser").select('div.card__content > div.data-listing')[2].select('span.data-listing__value')


[<span class="data-listing__value">30,23%</span>,
 <span class="data-listing__value">29,12%</span>,
 <span class="data-listing__value">196,96%</span>,
 <span class="data-listing__value">394,3 Mrd.</span>,
 <span class="data-listing__value">16,7 Mrd.</span>,
 <span class="data-listing__value">2.207.515</span>,
 <span class="data-listing__value">109,4 Mrd.</span>,
 <span class="data-listing__value">99,8 Mrd.</span>]